In [45]:
from sklearn.feature_selection import mutual_info_regression
import os
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import plotly.express as px
%matplotlib inline

sns.set_style('darkgrid')
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (10, 6)
matplotlib.rcParams['figure.facecolor'] = '#00000000'

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, confusion_matrix, classification_report)
from sklearn.exceptions import NotFittedError
from sklearn.impute import SimpleImputer

from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier,
                              GradientBoostingClassifier, BaggingClassifier, HistGradientBoostingClassifier, StackingClassifier)
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier

In [46]:
data=pd.read_csv('/kaggle/input/framingham-balanced/archive/CHD_preprocessed.csv')
data

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,1,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,1,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,1,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4128,1,50,0,1,1.0,0.0,0,1,0,313.0,179.0,92.0,25.97,66.0,86.0,1
4129,1,51,1,1,43.0,0.0,0,0,0,207.0,126.5,80.0,19.71,65.0,68.0,0
4130,0,48,0,1,20.0,0.0,0,0,0,248.0,131.0,72.0,22.00,84.0,86.0,0
4131,0,44,0,1,15.0,0.0,0,0,0,210.0,126.5,87.0,19.16,86.0,82.0,0


In [47]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4133 entries, 0 to 4132
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   male             4133 non-null   int64  
 1   age              4133 non-null   int64  
 2   education        4133 non-null   int64  
 3   currentSmoker    4133 non-null   int64  
 4   cigsPerDay       4133 non-null   float64
 5   BPMeds           4133 non-null   float64
 6   prevalentStroke  4133 non-null   int64  
 7   prevalentHyp     4133 non-null   int64  
 8   diabetes         4133 non-null   int64  
 9   totChol          4133 non-null   float64
 10  sysBP            4133 non-null   float64
 11  diaBP            4133 non-null   float64
 12  BMI              4133 non-null   float64
 13  heartRate        4133 non-null   float64
 14  glucose          4133 non-null   float64
 15  TenYearCHD       4133 non-null   int64  
dtypes: float64(8), int64(8)
memory usage: 516.8 KB


In [48]:
df= data.iloc[:4020]
count_0=df['TenYearCHD'].eq(0).sum()
count_1=df['TenYearCHD'].eq(1).sum()

print("count 0",count_0)
print("count 1",count_1)

count 0 3412
count 1 608


In [49]:
target= data['TenYearCHD']
columns_to_drop = ['TenYearCHD']
features = data.drop(columns_to_drop, axis=1)
features

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose
0,1,39,1,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0
1,0,46,0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0
2,1,48,0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0
3,0,61,1,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0
4,0,46,1,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4128,1,50,0,1,1.0,0.0,0,1,0,313.0,179.0,92.0,25.97,66.0,86.0
4129,1,51,1,1,43.0,0.0,0,0,0,207.0,126.5,80.0,19.71,65.0,68.0
4130,0,48,0,1,20.0,0.0,0,0,0,248.0,131.0,72.0,22.00,84.0,86.0
4131,0,44,0,1,15.0,0.0,0,0,0,210.0,126.5,87.0,19.16,86.0,82.0


# All undersampling

In [50]:
from imblearn.under_sampling import RandomUnderSampler, TomekLinks, ClusterCentroids, NearMiss, EditedNearestNeighbours

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

rus = RandomUnderSampler()
X_RUS, y_RUS = rus.fit_resample(X_train, y_train)

rf_model_rus = RandomForestClassifier(random_state=42)
rf_model_rus.fit(X_RUS, y_RUS)
y_pred_rus = rf_model_rus.predict(X_test)
print("Random Undersampling Accuracy:", accuracy_score(y_test, y_pred_rus))
print(classification_report(y_test, y_pred_rus))


tl = TomekLinks()
X_TL, y_TL = tl.fit_resample(X_train, y_train)

rf_model_tl = RandomForestClassifier(random_state=42)
rf_model_tl.fit(X_TL, y_TL)
y_pred_tl = rf_model_tl.predict(X_test)
print("\nTomek Links Accuracy:", accuracy_score(y_test, y_pred_tl))
print(classification_report(y_test, y_pred_tl))


cc = ClusterCentroids()
X_CC, y_CC = cc.fit_resample(X_train, y_train)

rf_model_cc = RandomForestClassifier(random_state=42)
rf_model_cc.fit(X_CC, y_CC)
y_pred_cc = rf_model_cc.predict(X_test)
print("\nCluster Centroids Accuracy:", accuracy_score(y_test, y_pred_cc))
print(classification_report(y_test, y_pred_cc))


nm = NearMiss()
X_NM, y_NM = nm.fit_resample(X_train, y_train)

rf_model_nm = RandomForestClassifier(random_state=42)
rf_model_nm.fit(X_NM, y_NM)
y_pred_nm = rf_model_nm.predict(X_test)
print("\nNearMiss Accuracy:", accuracy_score(y_test, y_pred_nm))
print(classification_report(y_test, y_pred_nm))


enn = EditedNearestNeighbours()
X_ENN, y_ENN = enn.fit_resample(X_train, y_train)

rf_model_enn = RandomForestClassifier(random_state=42)
rf_model_enn.fit(X_ENN, y_ENN)
y_pred_enn = rf_model_enn.predict(X_test)
print("\nEdited Nearest Neighbours Accuracy:", accuracy_score(y_test, y_pred_enn))
print(classification_report(y_test, y_pred_enn))


Random Undersampling Accuracy: 0.6481257557436517
              precision    recall  f1-score   support

           0       0.89      0.65      0.75       680
           1       0.28      0.64      0.39       147

    accuracy                           0.65       827
   macro avg       0.59      0.64      0.57       827
weighted avg       0.78      0.65      0.69       827


Tomek Links Accuracy: 0.8319226118500604
              precision    recall  f1-score   support

           0       0.84      0.99      0.91       680
           1       0.70      0.10      0.17       147

    accuracy                           0.83       827
   macro avg       0.77      0.54      0.54       827
weighted avg       0.81      0.83      0.78       827



/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(



Cluster Centroids Accuracy: 0.3385731559854897
              precision    recall  f1-score   support

           0       0.88      0.23      0.36       680
           1       0.19      0.85      0.31       147

    accuracy                           0.34       827
   macro avg       0.53      0.54      0.34       827
weighted avg       0.75      0.34      0.35       827


NearMiss Accuracy: 0.42926239419588874
              precision    recall  f1-score   support

           0       0.90      0.35      0.50       680
           1       0.21      0.82      0.34       147

    accuracy                           0.43       827
   macro avg       0.55      0.58      0.42       827
weighted avg       0.78      0.43      0.47       827


Edited Nearest Neighbours Accuracy: 0.8174123337363967
              precision    recall  f1-score   support

           0       0.85      0.94      0.89       680
           1       0.47      0.26      0.33       147

    accuracy                          

In [51]:
X_train

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose
173,0,60,1,0,0.0,0.0,0,1,0,325.0,182.0,106.0,27.61,80.0,77.0
1022,1,42,1,1,20.0,0.0,0,0,0,270.0,112.0,77.0,24.77,73.0,85.0
3182,1,58,0,0,0.0,0.0,0,0,0,225.0,105.5,74.0,25.68,50.0,93.0
331,0,58,0,0,0.0,0.0,0,1,0,200.0,158.0,101.0,23.06,85.0,77.0
2222,1,39,1,0,0.0,0.0,0,1,0,208.0,146.0,92.0,25.91,69.0,74.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3444,0,49,0,0,0.0,0.0,0,1,0,233.0,149.0,91.5,26.03,68.0,82.0
466,1,50,0,0,0.0,0.0,0,1,0,219.0,145.0,100.0,26.26,78.0,108.0
3092,0,36,0,0,0.0,0.0,0,0,0,209.0,107.0,73.5,21.59,75.0,73.0
3772,0,64,0,0,0.0,0.0,0,1,0,279.0,172.0,87.0,24.01,80.0,70.0


# SMOTH

In [52]:
from imblearn.over_sampling import SMOTE

target = data['TenYearCHD']
columns_to_drop = ['TenYearCHD']
features = data.drop(columns_to_drop, axis=1)

x = features.values
y = target.values
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

smote = SMOTE(random_state=42)
x_train, y_train = smote.fit_resample(x_train, y_train)

# model = RandomForestClassifier(random_state=42)
# model.fit(x_train, y_train)

# y_pred = model.predict(x_test)

# print("Classification Report:\n", classification_report(y_test, y_pred))

In [53]:
classifiers = {
    "RandomForest": RandomForestClassifier(random_state=42),
    "AdaBoost": AdaBoostClassifier(random_state=42),
    "ExtraTrees": ExtraTreesClassifier(n_estimators=1500, random_state=42),
    "GradientBoosting": GradientBoostingClassifier(random_state=42),
    "Bagging": BaggingClassifier(random_state=42),
    "HistGradientBoosting": HistGradientBoostingClassifier(random_state=42),
    "DecisionTree": DecisionTreeClassifier(random_state=42),
    "LogisticRegression": LogisticRegression(random_state=42),
    "GaussianNB": GaussianNB(),
    "BernoulliNB": BernoulliNB(),
    "MultinomialNB": MultinomialNB(),
    "KNeighbors": KNeighborsClassifier(),
    #"RadiusNeighbors": RadiusNeighborsClassifier(),
    "SVC": SVC(probability=True, random_state=42),
    "XGBoost": XGBClassifier(random_state=42),
    "LightGBM": LGBMClassifier(random_state=42),
    "MLPClassifier": MLPClassifier(random_state=42)
}

results_df = pd.DataFrame(columns=['Model', 'Accuracy', 'F1 Score', 'Precision', 'Recall', 'Specificity', 'ROC-AUC', 'Confusion Matrix'])

# def evaluate_classifiers(classifiers, x_train, x_test, y_train, y_test):
#     for name, clf in classifiers.items():
#         print(f"\nClassifier: {name}")
#         try:
#             clf.fit(x_train, y_train)
#             y_pred = clf.predict(x_test)

#             print("Classification Report:")
#             print(classification_report(y_test, y_pred))

#             cm = confusion_matrix(y_test, y_pred)
#             print("Confusion Matrix:")
#             print(cm)

#             acc = accuracy_score(y_test, y_pred)
#             print(f"Accuracy: {acc:.4f}")

#             f1 = f1_score(y_test, y_pred, average='weighted')
#             print(f"F1 Score: {f1:.4f}")

#             if hasattr(clf, "predict_proba"): 
#                 y_proba = clf.predict_proba(x_test)[:, 1]
#                 roc_auc = roc_auc_score(y_test, y_proba)
#                 print(f"ROC-AUC Score: {roc_auc:.4f}")
#         except (ValueError, NotFittedError) as e:
#             print(f"Error with classifier {name}: {e}")

# evaluate_classifiers(classifiers, x_train, x_test, y_train, y_test)

In [54]:
def evaluate_classifiers(classifiers, x_train, x_test, y_train, y_test):
    global results_df
    for name, clf in classifiers.items():
        try:
            clf.fit(x_train, y_train) 
            y_pred = clf.predict(x_test)  

            cm = confusion_matrix(y_test, y_pred)
            if len(cm.ravel()) == 4:  
                tn, fp, fn, tp = cm.ravel()
                specificity = tn / (tn + fp) 
            else:
                specificity = None

            acc = accuracy_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred, average='weighted')
            report = classification_report(y_test, y_pred, output_dict=True)
            precision = report['weighted avg']['precision']
            recall = report['weighted avg']['recall']

            if hasattr(clf, "predict_proba"): 
                y_proba = clf.predict_proba(x_test)[:, 1]
                roc_auc = roc_auc_score(y_test, y_proba)
            else:
                roc_auc = None
                
            new_row = pd.DataFrame({
                'Model': [name],
                'Accuracy': [acc],
                'F1 Score': [f1],
                'Precision': [precision],
                'Recall': [recall],
                'Specificity': [specificity],
                'ROC-AUC': [roc_auc],
                'Confusion Matrix': [cm]
            })
            results_df = pd.concat([results_df, new_row], ignore_index=True)

        except (ValueError, NotFittedError) as e:
            print(f"Error with classifier {name}: {e}")
            
evaluate_classifiers(classifiers, x_train, x_test, y_train, y_test)
print(results_df)

/tmp/ipykernel_36/3692562646.py:37: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, new_row], ignore_index=True)
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[LightGBM] [Info] Number of positive: 2825, number of negative: 2825
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001173 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3259
[LightGBM] [Info] Number of data points in the train set: 5650, number of used features: 15
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
                   Model  Accuracy  F1 Score  Precision    Recall  \
0           RandomForest  0.816203  0.772429   0.766545  0.816203   
1               AdaBoost  0.778718  0.767350   0.758510  0.778718   
2             ExtraTrees  0.814994  0.764340   0.757535  0.814994   
3       GradientBoosting  0.801693  0.767799   0.753688  0.801693   
4                Bagging  0.814994  0.776874   0.769310  0.814994   
5   HistGradientBoosting  0.808948  0.770226   0.758824  0.808948   
6           Decis

# TomekLinks SMOTH

In [55]:
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import TomekLinks

target = data['TenYearCHD']
columns_to_drop = ['TenYearCHD']
features = data.drop(columns_to_drop, axis=1)

x= features.values
y= target.values

tomeklinks= TomekLinks()
oversample= SMOTE()
steps= [("TL", tomeklinks), ("os", oversample)]
pipeline= Pipeline(steps=steps)

x_s, y_s = pipeline.fit_resample(x, y)

x_train, x_test,y_train, y_test = train_test_split(x_s, y_s, test_size=0.2, random_state=42)

# model= RandomForestClassifier(random_state=42)
# model.fit(x_train,y_train)

# y_pred = model.predict(x_test)
# print("Classification Report:\n",classification_report(y_test, y_pred))

In [56]:
y_counts = pd.Series(y_s).value_counts()
print(f"Count of 0: {y_counts[0]}")
print(f"Count of 1: {y_counts[1]}")

Count of 0: 3299
Count of 1: 3299


In [57]:
# from imblearn.over_sampling import SMOTE
# from imblearn.pipeline import Pipeline
# from imblearn.under_sampling import TomekLinks
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import train_test_split, GridSearchCV
# from sklearn.metrics import classification_report, accuracy_score

# # Define features and target arrays
# x = features.values
# y = target.values

# # Initialize Tomek Links and SMOTE with a more balanced sampling strategy
# tomeklinks = TomekLinks()
# smote = SMOTE(sampling_strategy=0.5)  # Reduce the amount of oversampling

# # Create a pipeline with Tomek Links and SMOTE
# steps = [
#     ("TL", tomeklinks),  # Remove noisy/borderline examples
#     ("smote", smote)     # Generate synthetic samples with SMOTE
# ]
# pipeline = Pipeline(steps=steps)

# # Apply the pipeline to resample the dataset
# x_s, y_s = pipeline.fit_resample(x, y)

# # Split the resampled data into training and testing sets
# x_train, x_test, y_train, y_test = train_test_split(x_s, y_s, test_size=0.2, random_state=42)

# # Initialize a classifier (e.g., RandomForest) and use GridSearchCV for hyperparameter tuning
# param_grid = {
#     'n_estimators': [100, 200, 300],
#     'max_depth': [5, 10, 20],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4]
# }

# rf = RandomForestClassifier(random_state=42)
# grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy')

# # Fit the model with the best parameters
# grid_search.fit(x_train, y_train)
# best_model = grid_search.best_estimator_

# # Predict on the test set
# y_pred = best_model.predict(x_test)

# # Evaluate the model performance
# print("Accuracy: ", accuracy_score(y_test, y_pred))
# print("Classification Report:\n", classification_report(y_test, y_pred))

In [58]:
classifiers = {
    "RandomForest": RandomForestClassifier(random_state=42),
    "AdaBoost": AdaBoostClassifier(random_state=42),
    "ExtraTrees": ExtraTreesClassifier(random_state=42),
    "GradientBoosting": GradientBoostingClassifier(random_state=42),
    "Bagging": BaggingClassifier(random_state=42),
    "HistGradientBoosting": HistGradientBoostingClassifier(random_state=42),
    "DecisionTree": DecisionTreeClassifier(random_state=42),
    "LogisticRegression": LogisticRegression(random_state=42),
    "GaussianNB": GaussianNB(),
    "BernoulliNB": BernoulliNB(),
    #"MultinomialNB": MultinomialNB(),
    "KNeighbors": KNeighborsClassifier(),
    #"RadiusNeighbors": RadiusNeighborsClassifier(),
    "SVC": SVC(probability=True, random_state=42),
    "XGBoost": XGBClassifier(random_state=42),
    "LightGBM": LGBMClassifier(random_state=42),
    "MLPClassifier": MLPClassifier(random_state=42)
}
results_df = pd.DataFrame(columns=['Model', 'Accuracy', 'F1 Score', 'Precision', 'Recall', 'Specificity', 'ROC-AUC', 'Confusion Matrix'])


def evaluate_classifiers(classifiers, x_train, x_test, y_train, y_test):
    global results_df
    for name, clf in classifiers.items():
        try:
            clf.fit(x_train, y_train) 
            y_pred = clf.predict(x_test)  

            cm = confusion_matrix(y_test, y_pred)
            if len(cm.ravel()) == 4:  
                tn, fp, fn, tp = cm.ravel()
                specificity = tn / (tn + fp) 
            else:
                specificity = None

            acc = accuracy_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred, average='weighted')
            report = classification_report(y_test, y_pred, output_dict=True)
            precision = report['weighted avg']['precision']
            recall = report['weighted avg']['recall']

            if hasattr(clf, "predict_proba"): 
                y_proba = clf.predict_proba(x_test)[:, 1]
                roc_auc = roc_auc_score(y_test, y_proba)
            else:
                roc_auc = None
                
            new_row = pd.DataFrame({
                'Model': [name],
                'Accuracy': [acc],
                'F1 Score': [f1],
                'Precision': [precision],
                'Recall': [recall],
                'Specificity': [specificity],
                'ROC-AUC': [roc_auc],
                'Confusion Matrix': [cm]
            })
            results_df = pd.concat([results_df, new_row], ignore_index=True)

        except (ValueError, NotFittedError) as e:
            print(f"Error with classifier {name}: {e}")
            
evaluate_classifiers(classifiers, x_train, x_test, y_train, y_test)
print(results_df)

/tmp/ipykernel_36/1245361844.py:59: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, new_row], ignore_index=True)
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[LightGBM] [Info] Number of positive: 2660, number of negative: 2618
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001114 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3243
[LightGBM] [Info] Number of data points in the train set: 5278, number of used features: 15
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503979 -> initscore=0.015915
[LightGBM] [Info] Start training from score 0.015915
                   Model  Accuracy  F1 Score  Precision    Recall  \
0           RandomForest  0.915909  0.915779   0.916944  0.915909   
1               AdaBoost  0.807576  0.807096   0.808770  0.807576   
2             ExtraTrees  0.937121  0.937067   0.937584  0.937121   
3       GradientBoosting  0.871212  0.870186   0.879043  0.871212   
4                Bagging  0.893939  0.893658   0.895968  0.893939   
5   HistGradientBoosting  0.89393

In [64]:
exc= ExtraTreesClassifier(criterion='entropy', n_estimators= 100, max_depth= 500, min_samples_split= 3, max_features='log2')
exc.fit(x_train,y_train)
y_pred= exc.predict(x_test)
print("classification report: \n", classification_report(y_test, y_pred))

classification report: 
               precision    recall  f1-score   support

           0       0.92      0.96      0.94       681
           1       0.95      0.92      0.93       639

    accuracy                           0.94      1320
   macro avg       0.94      0.94      0.94      1320
weighted avg       0.94      0.94      0.94      1320



# mlp

In [60]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

model = Sequential([
    Dense(128, activation='relu', input_shape=(x_train.shape[1],)),  
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),   
    Dense(1, activation='sigmoid')                                
])

optimizer = Adam(learning_rate=0.02)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
model.fit(x_train_scaled, y_train, epochs=100, batch_size=32, validation_data=(x_test_scaled, y_test))
test_loss, test_accuracy = model.evaluate(x_test_scaled, y_test)
print(f"Test Accuracy: {test_accuracy:.4f}")

KeyboardInterrupt: 

# Stacked Ensemble (SVM, DT, XGBoost, LR as meta-classifier)

In [ ]:
base_estimators = [
    ('svm', SVC(probability=True, kernel='linear', random_state=42)),
    #('et', ExtraTreesClassifier(n_estimators=100, random_state=42)),
    ('dt', DecisionTreeClassifier(random_state=42)),                   
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)) 
]
meta_learner = LogisticRegression()
stacking_clf = StackingClassifier(estimators=base_estimators, final_estimator=meta_learner, cv=5)
stacking_clf.fit(x_train, y_train)

y_pred = stacking_clf.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy)
print("Classification Report:\n", classification_report(y_test, y_pred))

In [ ]:
base_estimators = [
    ('et', ExtraTreesClassifier(n_estimators=100, random_state=42)),
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42))
]

# মেটা-মডেল (স্ট্যাকার) হিসেবে LogisticRegression
stacking_clf = StackingClassifier(
    estimators=base_estimators, 
    final_estimator=LogisticRegression(),
    cv=5
)

model_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')), 
    ('scaler', StandardScaler()),               
    ('stacking', stacking_clf)                   
])
model_pipeline.fit(x_train, y_train)
y_pred = model_pipeline.predict(x_test)

print("Classification Report:\n", classification_report(y_test, y_pred))

In [ ]:
# y_pred = model.predict(x_test)
# print("Classification Report:\n",classification_report(y_test, y_pred))

# Stacked Sparse Autoencoder (SSAE) + PSO built this model

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.constraints import max_norm

x = features.values
y = target.values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

def build_autoencoder(input_dim, encoding_dim, sparsity_weight=0.01):
    # Input Layer
    input_layer = Input(shape=(input_dim,))
    
    # Encoding Layers
    encoded = Dense(encoding_dim, activation='relu', activity_regularizer=tf.keras.regularizers.l1(sparsity_weight))(input_layer)
    
    # Decoding Layers
    decoded = Dense(input_dim, activation='sigmoid')(encoded)
    
    # Autoencoder Model
    autoencoder = Model(input_layer, decoded)
    
    # Encoder Model
    encoder = Model(input_layer, encoded)
    
    return autoencoder, encoder

input_dim = x_train_scaled.shape[1]
encoding_dims = [128, 64]  # Example dimensions for stacking

# Build Stacked Autoencoder
autoencoder, encoder = build_autoencoder(input_dim, encoding_dims[0])

# Compile the model
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Train the model
autoencoder.fit(x_train_scaled, x_train_scaled, epochs=50, batch_size=256, shuffle=True, validation_data=(x_test_scaled, x_test_scaled))

In [ ]:
pip install pyswarm

In [ ]:
from pyswarm import pso
import numpy as np

def objective_function(params):
    encoding_dim1, encoding_dim2 = int(params[0]), int(params[1])
    autoencoder, encoder = build_autoencoder(input_dim, encoding_dim1)
    autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
    
    history = autoencoder.fit(x_train_scaled, x_train_scaled, epochs=50, batch_size=256, shuffle=True, validation_data=(x_test_scaled, x_test_scaled), verbose=0)
    
    val_loss = history.history['val_loss'][-1]
    return val_loss

lb = [10, 10]  # Lower bounds for encoding dimensions
ub = [256, 128]  # Upper bounds for encoding dimensions

best_params, _ = pso(objective_function, lb, ub, swarmsize=10, maxiter=5)
print("Best Parameters:", best_params)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import SMOTE
from pyswarm import pso


target = data['TenYearCHD']
columns_to_drop = ['TenYearCHD']
features = data.drop(columns_to_drop, axis=1)

x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

def build_autoencoder(input_dim, encoding_dim1, encoding_dim2, sparsity_weight=0.01):
    input_layer = Input(shape=(input_dim,))
    encoded1 = Dense(encoding_dim1, activation='relu', activity_regularizer=tf.keras.regularizers.l1(sparsity_weight))(input_layer)
    encoded2 = Dense(encoding_dim2, activation='relu', activity_regularizer=tf.keras.regularizers.l1(sparsity_weight))(encoded1)
    decoded = Dense(input_dim, activation='sigmoid')(encoded2)
    autoencoder = Model(input_layer, decoded)
    encoder = Model(input_layer, encoded2)
    return autoencoder, encoder

encoding_dim1 = 72
encoding_dim2 = 128

input_dim = x_train_scaled.shape[1]
autoencoder, encoder = build_autoencoder(input_dim, encoding_dim1, encoding_dim2)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.fit(x_train_scaled, x_train_scaled, epochs=50, batch_size=256, shuffle=True, validation_data=(x_test_scaled, x_test_scaled))

reconstructed = autoencoder.predict(x_test_scaled)
test_loss = autoencoder.evaluate(x_test_scaled, x_test_scaled)
print(f"Test Loss: {test_loss:.4f}")

encoded_data = encoder.predict(x_test_scaled)
print(f"Encoded Data Shape: {encoded_data.shape}")


# Borderline-SMOTE + TPE + LightGBM 

In [ ]:
from imblearn.over_sampling import BorderlineSMOTE
from lightgbm import LGBMClassifier

x = features.values
y = target.values
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

borderline_smote = BorderlineSMOTE()
lgbm_model = LGBMClassifier()

pipeline = Pipeline([
    ('smote', borderline_smote),
    ('lgbm', lgbm_model)
])

space = {
    'lgbm__num_leaves': hp.choice('num_leaves', [31, 63, 127, 255]),
    'lgbm__learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
    'lgbm__n_estimators': hp.choice('n_estimators', [50, 100, 200, 300])
}

def objective(params):
    pipeline.set_params(**params)
    pipeline.fit(x_train_scaled, y_train)
    score = pipeline.score(x_test_scaled, y_test)
    return {'loss': -score, 'status': 'ok'}

trials = Trials()
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=50, trials=trials)

num_leaves_options = [31, 63, 127, 255]
n_estimators_options = [50, 100, 200, 300]

best_params = {
    'lgbm__num_leaves': num_leaves_options[best['num_leaves']],  
    'lgbm__learning_rate': best['learning_rate'], 
    'lgbm__n_estimators': n_estimators_options[best['n_estimators']] 
}

print("Best parameters found: ", best_params)

pipeline.set_params(**best_params)
pipeline.fit(x_train_scaled, y_train)

test_score = pipeline.score(x_test_scaled, y_test)
print(f"Test Accuracy with best parameters: {test_score:.4f}")

# Borderline-tome-SMOTE + TPE + LightGBM  built this model

In [ ]:
import lightgbm as lgb
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import optuna
import numpy as np
import pandas as pd

# Sample Data Loading (Replace with your own data loading code)
# features = pd.read_csv('your_features.csv')
# target = pd.read_csv('your_target.csv')

# For demonstration, we use random data
np.random.seed(42)
features = pd.DataFrame(np.random.rand(1000, 20), columns=[f'feature_{i}' for i in range(20)])
target = pd.Series(np.random.randint(0, 2, size=1000), name='target')

# Prepare data
x = features.values
y = target.values

# Split the dataset
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Define preprocessing pipeline with TomekLinks and SMOTE
tomeklinks = TomekLinks()
smote = SMOTE()
pipeline = Pipeline(steps=[("TL", tomeklinks), ("SMOTE", smote)])

# Apply the pipeline to the training data
x_train_resampled, y_train_resampled = pipeline.fit_resample(x_train, y_train)

# Define the objective function for Optuna
def objective(trial):
    param = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 31, 255),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'max_depth': trial.suggest_int('max_depth', -1, 50)
    }
    
    # Train LightGBM model
    model = lgb.LGBMClassifier(**param)
    model.fit(x_train_resampled, y_train_resampled)
    
    # Evaluate the model
    y_pred = model.predict(x_test)
    accuracy = (y_pred == y_test).mean()
    return accuracy

# Create Optuna study object
study = optuna.create_study(direction='maximize')

# Optimize the objective function
study.optimize(objective, n_trials=50)

# Print the best hyperparameters
print("Best hyperparameters: ", study.best_params)

# Train the final model with the best hyperparameters
best_params = study.best_params
model = lgb.LGBMClassifier(**best_params)
model.fit(x_train_resampled, y_train_resampled)

# Make predictions and evaluate the final model
y_pred = model.predict(x_test)
print("Classification Report:\n", classification_report(y_test, y_pred))


# GAN

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

In [ ]:
def build_generator(noise_dim):
    model = tf.keras.Sequential()
    
#     model.add(layers.Dense(16, input_dim= noise_dim))
#     model.add(layers.LeakyReLU(alpha=0.2))
#     model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Dense(32, input_dim= noise_dim),)
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Dense(64))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization(momentum=0.8))
    
    #model.add(layers.Dense(128, input_dim=noise_dim))
    model.add(layers.Dense(128))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Dense(256))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Dense(512))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Dense(1024))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Dense(np.prod(features.shape[1:]), activation='tanh'))
    model.add(layers.Reshape(features.shape[1:]))
    
    return model

In [ ]:
def build_discriminator(data_shape):
    model= tf.keras.Sequential()
    model.add(layers.Flatten(input_shape= data_shape))
    model.add(layers.Dense(512))
    model.add(layers.LeakyReLU(alpha= 0.2))
    model.add(layers.Dense(256))
    model.add(layers.LeakyReLU(alpha= 0.2))
    model.add(layers.Dense(128))
    model.add(layers.LeakyReLU(alpha= 0.2))
    
    model.add(layers.Dense(64))
    model.add(layers.LeakyReLU(alpha= 0.2))
    model.add(layers.Dense(32))
    model.add(layers.LeakyReLU(alpha= 0.2))
    
    model.add(layers.Dense(1, activation= 'sigmoid'))
    
    return model

In [ ]:
noise_dim= 100
generator= build_generator(noise_dim)
discriminator= build_discriminator(features.shape[1:])
discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# GAN model
discriminator.trainable = False
gan_input = layers.Input(shape=(noise_dim,))
generated_data = generator(gan_input)
gan_output = discriminator(generated_data)

gan = tf.keras.Model(gan_input, gan_output)
gan.compile(loss='binary_crossentropy', optimizer='adam')

In [ ]:
def train_gan(gan, generator, discriminator, features, noise_dim, epochs=100, batch_size=64):
    for epoch in range(epochs):
        # Train the discriminator
        idx = np.random.randint(0, features.shape[0], batch_size)
        real_data = features.iloc[idx]  # Use .iloc for integer-based indexing
        
        noise = np.random.normal(0, 1, (batch_size, noise_dim))
        generated_data = generator.predict(noise)
        
        d_loss_real = discriminator.train_on_batch(real_data, np.ones((batch_size, 1)))
        d_loss_fake = discriminator.train_on_batch(generated_data, np.zeros((batch_size, 1)))
        
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        
        # Train the generator
        noise = np.random.normal(0, 1, (batch_size, noise_dim))
        valid_y = np.array([1] * batch_size)
        g_loss = gan.train_on_batch(noise, valid_y)

        if epoch % 100 == 0:
            print(f"Epoch {epoch}/{epochs} [D loss: {d_loss[0]}, acc.: {100*d_loss[1]:.2f}] [G loss: {g_loss}]")

train_gan(gan, generator, discriminator, features, noise_dim, epochs=100, batch_size=64)

In [ ]:
# def train_gan(gan, generator, discriminator, features, noise_dim, epochs=100, batch_size=64):
#     for epoch in range(epochs):
#         idx = np.random.randint(0, features.shape[0], batch_size)
#         real_data = features.iloc[idx]  
#         noise = np.random.normal(0, 1, (batch_size, noise_dim))
#         generated_data = generator.predict(noise)

#         real_labels = np.ones((batch_size, 1))
#         fake_labels = np.zeros((batch_size, 1))
        
#         d_loss_real, d_acc_real = discriminator.train_on_batch(real_data, real_labels)
#         d_loss_fake, d_acc_fake = discriminator.train_on_batch(generated_data, fake_labels)
        
#         d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        
#         real_preds = discriminator.predict(real_data)
#         fake_preds = discriminator.predict(generated_data)
        
#         real_preds_binary = (real_preds > 0.5).astype(int)
#         fake_preds_binary = (fake_preds > 0.5).astype(int)
        
#         true_labels_real = np.ones(real_preds_binary.shape)
#         true_labels_fake = np.zeros(fake_preds_binary.shape)
        
#         cm_real = confusion_matrix(true_labels_real, real_preds_binary)
#         cm_fake = confusion_matrix(true_labels_fake, fake_preds_binary)
        
#         tn_real, fp_real, fn_real, tp_real = cm_real.ravel() if cm_real.size == 4 else (0, 0, 0, 0)
#         tn_fake, fp_fake, fn_fake, tp_fake = cm_fake.ravel() if cm_fake.size == 4 else (0, 0, 0, 0)
        
#         total_fp = fp_real + fp_fake
#         total_fn = fn_real + fn_fake
#         total_tp = tp_real + tp_fake
#         total_tn = tn_real + tn_fake

#         noise = np.random.normal(0, 1, (batch_size, noise_dim))
#         valid_y = np.array([1] * batch_size)
#         g_loss = gan.train_on_batch(noise, valid_y)

#         if epoch % 100 == 0:
#             print(f"Epoch {epoch}/{epochs} [D loss: {d_loss}, acc.: {100*d_acc_real:.2f}] [G loss: {g_loss}]")
#             print(f"True Positives: {total_tp}")
#             print(f"False Positives: {total_fp}")
#             print(f"True Negatives: {total_tn}")
#             print(f"False Negatives: {total_fn}")
# train_gan(gan, generator, discriminator, features, noise_dim, epochs=100, batch_size=64)

In [ ]:
num_generated_samples = 2800
# Generate synthetic data
noise = np.random.normal(0, 1, (num_generated_samples, noise_dim))
generated_data = generator.predict(noise)

combined_data = np.concatenate([features, generated_data], axis=0)
combined_targets = np.concatenate([target, np.ones(num_generated_samples)], axis=0)  #synthetic data targets are 1

In [ ]:
count_zeros = np.sum(combined_targets == 0)
count_ones = np.sum(combined_targets == 1)

print(f"Count of 0s (No Heart Attack): {count_zeros}")
print(f"Count of 1s (Heart Attack): {count_ones}")

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(combined_data, combined_targets, test_size=0.2, random_state=42)

print('Train shape:', x_train.shape)
print('Test shape:', x_test.shape)

In [ ]:
# import lightgbm as lgb
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# lgbm = lgb.LGBMClassifier(
#     boosting_type='dart',
#     learning_rate=0.05,
#     n_estimators=40,
#     num_leaves=31,
#     random_state=42
# )
# lgbm.fit(x_train, y_train)

# y_pred = lgbm.predict(x_test)
# y_prob = lgbm.predict_proba(x_test)[:, 1]

# accuracy = accuracy_score(y_test, y_pred)
# precision = precision_score(y_test, y_pred)
# recall = recall_score(y_test, y_pred)
# f1 = f1_score(y_test, y_pred)
# roc_auc = roc_auc_score(y_test, y_prob)

# print(f'Accuracy: {accuracy:.4f}')
# print(f'Precision: {precision:.4f}')
# print(f'Recall: {recall:.4f}')
# print(f'F1 Score: {f1:.4f}')
# print(f'ROC AUC Score: {roc_auc:.4f}')

In [ ]:
# def evaluate_model(model, x_train, y_train, x_test, y_test):
#     # Predictions
#     y_pred_train = model.predict(x_train)
#     y_pred_test = model.predict(x_test)
    
#     accuracy_train = accuracy_score(y_train, y_pred_train)
#     accuracy_test = accuracy_score(y_test, y_pred_test)
#     precision_test = precision_score(y_test, y_pred_test)
#     recall_test = recall_score(y_test, y_pred_test)
#     f1_test = f1_score(y_test, y_pred_test)
#     auc = roc_auc_score(y_test, model.predict_proba(x_test)[:, 1])
#     mcc_test = matthews_corrcoef(y_test, y_pred_test)
#     kappa_test = cohen_kappa_score(y_test, y_pred_test)

#     tn, fp, fn, tp = confusion_matrix(y_test, y_pred_test).ravel()
#     specificity_test = tn / (tn + fp)
#     sensitivity_test = recall_test  
    
#     print(f"Train Accuracy: {accuracy_train*100:.2f}%")
#     print(f"Test Accuracy: {accuracy_test*100:.2f}%")
#     print(f"Precision: {precision_test*100:.2f}%")
#     print(f"Recall/Sensitivity: {sensitivity_test*100:.2f}%")
#     print(f"Specificity: {specificity_test*100:.2f}%")
#     print(f"F1-Score: {f1_test*100:.2f}%")
#     print(f"AUC-ROC: {auc*100:.2f}%")
#     print(f"MCC: {mcc_test*100:.2f}%")
#     print(f"Kappa: {kappa_test*100:.2f}%")

In [ ]:
classifiers = {
    "RandomForest": RandomForestClassifier(random_state=42),
    "AdaBoost": AdaBoostClassifier(random_state=42),
    "ExtraTrees": ExtraTreesClassifier(random_state=42),
    "GradientBoosting": GradientBoostingClassifier(random_state=42),
    "Bagging": BaggingClassifier(random_state=42),
    "HistGradientBoosting": HistGradientBoostingClassifier(random_state=42),
    "DecisionTree": DecisionTreeClassifier(random_state=42),
    "LogisticRegression": LogisticRegression(random_state=42),
    "GaussianNB": GaussianNB(),
    "BernoulliNB": BernoulliNB(),
    #"MultinomialNB": MultinomialNB(),
    "KNeighbors": KNeighborsClassifier(),
    #"RadiusNeighbors": RadiusNeighborsClassifier(),
    "SVC": SVC(probability=True, random_state=42),
    "XGBoost": XGBClassifier(random_state=42),
    "LightGBM": LGBMClassifier(random_state=42),
    "MLPClassifier": MLPClassifier(random_state=42)
}
results_df = pd.DataFrame(columns=['Model', 'Accuracy', 'F1 Score', 'Precision', 'Recall', 'Specificity', 'ROC-AUC', 'Confusion Matrix'])


def evaluate_classifiers(classifiers, x_train, x_test, y_train, y_test):
    global results_df
    for name, clf in classifiers.items():
        try:
            clf.fit(x_train, y_train) 
            y_pred = clf.predict(x_test)  

            cm = confusion_matrix(y_test, y_pred)
            if len(cm.ravel()) == 4:  
                tn, fp, fn, tp = cm.ravel()
                specificity = tn / (tn + fp) 
            else:
                specificity = None

            acc = accuracy_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred, average='weighted')
            report = classification_report(y_test, y_pred, output_dict=True)
            precision = report['weighted avg']['precision']
            recall = report['weighted avg']['recall']

            if hasattr(clf, "predict_proba"): 
                y_proba = clf.predict_proba(x_test)[:, 1]
                roc_auc = roc_auc_score(y_test, y_proba)
            else:
                roc_auc = None
                
            new_row = pd.DataFrame({
                'Model': [name],
                'Accuracy': [acc],
                'F1 Score': [f1],
                'Precision': [precision],
                'Recall': [recall],
                'Specificity': [specificity],
                'ROC-AUC': [roc_auc],
                'Confusion Matrix': [cm]
            })
            results_df = pd.concat([results_df, new_row], ignore_index=True)

        except (ValueError, NotFittedError) as e:
            print(f"Error with classifier {name}: {e}")
            
evaluate_classifiers(classifiers, x_train, x_test, y_train, y_test)
print(results_df)

In [ ]:
exc= ExtraTreesClassifier(criterion='entropy', n_estimators= 100, max_depth= 500, min_samples_split= 2, max_features='log2')
exc.fit(x_train,y_train)
y_pred= exc.predict(x_test)
print("classification report: \n", classification_report(y_test, y_pred))

In [ ]:
from sklearn.linear_model import LogisticRegression
#lr = LogisticRegression(solver='newton-cg',multi_class='multinomial') #0.8561320754716981
#lr =LogisticRegression(solver='saga', multi_class='multinomial') #0.8561320754716981
lr = LogisticRegression(solver='liblinear',multi_class='ovr', C=0.1)
#lr.fit(x_train_resampled, y_train_resampled) #fit(x_train, y_train)
lr.fit(x_train, y_train)
print("\n")
print("Classification Report:\n", classification_report(y_test, y_pred))

# import shap
# explainer = shap.Explainer(lr, x_train)
# shap_values = explainer.shap_values(x_test[:100])
# plt.figure(figsize=(16, 12))
# shap.summary_plot(shap_values, features=x_test[:100], feature_names=features.columns)
# plt.show()

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score

sgd_clf = make_pipeline(StandardScaler(), SGDClassifier(max_iter=5000, random_state=500))
#sgd_clf.fit(x_train_resampled, y_train_resampled)  #fit(x_train, y_train)
sgd_clf.fit(x_train, y_train)

y_pred = sgd_clf.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy: {:.2f}%".format(accuracy * 100))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
#model=RandomForestClassifier(n_jobs=-1,random_state=42,max_features=2,min_samples_split=80, min_samples_leaf=50,bootstrap=False)
model=RandomForestClassifier(class_weight='balanced')
model.fit(x_train,y_train)
model.fit(x_train,y_train)
evaluate_model(model, x_train, y_train, x_test, y_test)

In [ ]:
model=RandomForestClassifier(n_jobs=-1,random_state=42,n_estimators=5,max_depth=7,max_leaf_nodes=4,
                             max_features=32).fit(x_train, y_train)
evaluate_model(model, x_train, y_train, x_test, y_test)